In [1]:
import os
import sys
import pickle
import pandas as pd
import streamlit as st
sys.path.append(os.path.abspath('..'))
from utilityfunctions.explaination import Explainer
from utilityfunctions.recommendation import Recommender

# ----------------------------
# Load trained bundle
# ----------------------------
with open("models/hybrid_bundle.pkl", "rb") as f:
    bundle = pickle.load(f)

recommender = Recommender(bundle, alpha=0.6)
explainer = Explainer(bundle)

movies_df = bundle['movies']  # assuming dict bundle

# ----------------------------
# Streamlit UI
# ----------------------------
st.set_page_config(page_title="Movie Recommender", layout="wide")
st.title("🎬 Hybrid Movie Recommender")

st.sidebar.header("User Preferences")
user_id = st.sidebar.number_input("Your User ID:", min_value=1, value=1)

# ----------------------------
# Section 1: Rate a Movie
# ----------------------------
st.header("1️⃣ Rate a Movie")

selected_movie = st.selectbox("Select a movie to rate:", movies_df['title'])
rating = st.slider("Your rating:", 0.5, 5.0, 3.0, 0.5)

if st.button("Submit Rating"):
    # Get movieid from title
    movie_id = movies_df.loc[movies_df['title'] == selected_movie, 'movieid'].values[0]
    user_seen = [movie_id]

    # Recommend similar movies based on rated movie
    recs = recommender.recommend(user_id, user_seen_movies=user_seen, k=10)
    st.subheader(f"Movies similar to '{selected_movie}'")
    for _, row in recs.iterrows():
        st.write(f"🎬 {row['title']} — Genres: {row['genre']} — Score: {row['hybrid_score']:.3f}")

# ----------------------------
# Section 2: Search by Genre
# ----------------------------
st.header("2️⃣ Search Movies by Genre")

all_genres = sorted({g for gs in movies_df['genre'] for g in gs.split('|')})
selected_genre = st.multiselect("Select genres:", all_genres)

if st.button("Search by Genre"):
    if selected_genre:
        # Filter movies matching selected genres
        filtered_movies = movies_df[movies_df['genre'].apply(lambda x: any(g in x.split('|') for g in selected_genre))]
        filtered_ids = filtered_movies['movieid'].tolist()
        # Recommend based on filtered movies
        recs = recommender.recommend(user_id, user_seen_movies=[], k=10)
        # Filter recommendations to match genre
        recs = recs[recs['movieid'].isin(filtered_ids)]
        st.subheader(f"Recommended movies for genre(s): {', '.join(selected_genre)}")
        for _, row in recs.iterrows():
            st.write(f"🎬 {row['title']} — Genres: {row['genre']} — Score: {row['hybrid_score']:.3f}")
    else:
        st.warning("Please select at least one genre!")

ModuleNotFoundError: No module named 'streamlit'